In [1]:
import pandas as pd 
import numpy as np
import xgboost as xgb
from sklearn import metrics
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import pickle
import json
import glob
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
root = '../'
X_train = pd.read_csv(root + 'data/Haleh/feature_datasets/X_train.csv')
X_train_full = pd.read_csv(root + 'data/Haleh/feature_datasets/X_train_dec.csv')
X_test = pd.read_csv(root + 'data/Haleh/feature_datasets/X_test.csv')
X_test_full = pd.read_csv(root + 'data/Haleh/feature_datasets/X_test_dec.csv')

y_train = np.load(root + 'data/Haleh/y_train.npy')
y_test = np.load(root + 'data/Haleh/y_test.npy')

In [3]:
X_train_full.head()

,z_0,z_1,z_2,z_3,z_4,z_5,z_6,z_7,z_8,z_9,...,0__fourier_entropy__bins_2,0__fourier_entropy__bins_3,0__fourier_entropy__bins_5,0__fourier_entropy__bins_10,0__fourier_entropy__bins_100,0__permutation_entropy__dimension_3__tau_1,0__permutation_entropy__dimension_4__tau_1,0__permutation_entropy__dimension_5__tau_1,0__permutation_entropy__dimension_6__tau_1,0__permutation_entropy__dimension_7__tau_1
0,-4.102828,12.117376,2.979742,6.824316,8.521718,-3.394531,3.834200,-5.100700,-0.607759,0.902113,...,0.079983,0.079983,0.170467,0.280179,0.597595,1.203390,1.766804,2.354477,2.933702,3.466107
1,-6.215706,12.255225,22.190172,-0.212288,-5.466818,-28.439009,-11.580964,11.495201,-4.141428,10.182698,...,0.163982,0.247940,0.327050,0.488548,0.796679,1.198716,1.766011,2.367617,2.970901,3.556645
2,-6.349346,12.251351,22.159270,-0.396654,-5.702261,-28.601473,-11.648271,11.519348,-4.056820,10.192198,...,0.210883,0.275463,0.408537,0.482843,0.907649,1.201300,1.770794,2.372899,2.981178,3.582805
3,-1.180088,-9.493090,12.479339,2.376886,3.711802,-5.103728,0.372605,11.038977,-3.201739,-4.864267,...,0.110453,0.155665,0.280179,0.452701,0.859510,1.256123,1.882432,2.550049,3.222475,3.874065
4,-1.003413,-10.825782,13.547513,2.483605,4.324685,-5.714592,0.728139,12.329692,-3.472347,-5.438126,...,0.110453,0.159721,0.235155,0.412056,0.741638,1.240362,1.833065,2.450065,3.049852,3.636782


In [22]:
best_params = {'DEC_Feature_DS': {
    'gblinear': {'alpha': 1.1375256185723135e-08,
                 'booster': 'gblinear',
                 'feature_selector': 'shuffle',
                 'lambda': 3.6963750283518966e-08,
                 'learning_rate': 0.3182599985270911,
                 'num_boost_round': 729,
                 'objective': 'binary:logistic',
                 'scale_pos_weight': 6,
                 'updater': 'shotgun',
                 'verbosity': 0}},
               'Feature_DS': {
    'gblinear': {'alpha': 0.05663610210741058,
                 'booster': 'gblinear',
                 'feature_selector': 'shuffle',
                 'lambda': 9.25194196329593e-08,
                 'learning_rate': 0.0405063980023227,
                 'num_boost_round': 617,
                 'objective': 'binary:logistic',
                 'scale_pos_weight': 6,
                 'updater': 'shotgun',
                 'verbosity': 0}}}

In [6]:
space={'alpha': hp.uniform("alpha", 0, 12),
        'lambda': hp.uniform('lambda', 0,12),
        'learning_rate' : hp.uniform('learning_rate', 0.01, 0.5),
        'num_boost_round' : hp.uniform('num_boost_round', 5, 1000),
        'objective': 'binary:logistic',
        'scale_pos_weight': 1,
        'updater': 'shotgun',
        'verbosity': 0,
        'booster': 'gblinear',
        'feature_selector': 'shuffle',
        'n_estimators': 180
    }

In [7]:
def objective_dec(space):
    params = {
        'alpha': space['alpha'],
        'lambda': space['lambda'],
        'learning_rate' : space['learning_rate'],
        'num_boost_round' : int(space['num_boost_round']),
        'objective': space['objective'],
        'scale_pos_weight': space['scale_pos_weight'],
        'updater': space['updater'],
        'verbosity': space['verbosity'],
        'booster': space['booster'],
        'feature_selector': space['feature_selector'],
        'n_estimators': space['n_estimators']
    }
    
    n_splits = 3
    skf = StratifiedKFold(n_splits)
    
    auc = 0.0
    for train_index, eval_index in skf.split(X_train_full, y_train):
        x_train_split = X_train_full[X_train_full.index.isin(train_index)]
        x_eval_split = X_train_full[X_train_full.index.isin(eval_index)]
        y_train_split = y_train[train_index]
        y_eval_split = y_train[eval_index]
        
        clf=xgb.XGBClassifier(**params, use_label_encoder=False)

        evaluation = [(x_train_split, y_train_split), (x_eval_split, y_eval_split)]

        clf.fit(x_train_split, y_train_split,
                eval_set=evaluation, eval_metric="auc",
                early_stopping_rounds=12,verbose=False)


        pred = clf.predict_proba(x_eval_split)[:, 1]
        auc += metrics.roc_auc_score(y_eval_split, pred)
        
    auc /= n_splits
    print (f"SCORE: \t\t{auc:.3f}")
    return {'loss': -auc, 'status': STATUS_OK }

def objective(space):
    params = {
        'alpha': space['alpha'],
        'lambda': space['lambda'],
        'learning_rate' : space['learning_rate'],
        'num_boost_round' : int(space['num_boost_round']),
        'objective': space['objective'],
        'scale_pos_weight': space['scale_pos_weight'],
        'updater': space['updater'],
        'verbosity': space['verbosity'],
        'booster': space['booster'],
        'feature_selector': space['feature_selector'],
        'n_estimators': space['n_estimators']
    }
    
    n_splits = 3
    skf = StratifiedKFold(n_splits)
    auc = 0.0
    for train_index, eval_index in skf.split(X_train, y_train):
        x_train_split = X_train[X_train.index.isin(train_index)]
        x_eval_split = X_train[X_train.index.isin(eval_index)]
        y_train_split = y_train[train_index]
        y_eval_split = y_train[eval_index]
        
        clf=xgb.XGBClassifier(**params, use_label_encoder=False)
    
        evaluation = [(x_train_split, y_train_split), (x_eval_split, y_eval_split)]
    
        clf.fit(x_train_split, y_train_split,
                eval_set=evaluation, eval_metric="auc",
                early_stopping_rounds=10,verbose=False)


        pred = clf.predict_proba(x_eval_split)[:, 1]
        auc += metrics.roc_auc_score(y_eval_split, pred)
        
    auc /= n_splits

    print (f"SCORE: \t\t{auc:.3f}")
    return {'loss': -auc, 'status': STATUS_OK }

In [8]:
trials_standard = Trials()

best_hyperparams_standard = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 1000,
                        trials = trials_standard)

SCORE: 		0.625                                                                                                                                                            
SCORE: 		0.560                                                                                                                                                            
SCORE: 		0.536                                                                                                                                                            
SCORE: 		0.631                                                                                                                                                            
SCORE: 		0.633                                                                                                                                                            
SCORE: 		0.529                                                                                                                                   

In [9]:
trials_dec = Trials()

best_hyperparams_dec = fmin(fn = objective_dec,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 1000,
                        trials = trials_dec)

SCORE: 		0.542                                                                                                                                                            
SCORE: 		0.577                                                                                                                                                            
SCORE: 		0.551                                                                                                                                                            
SCORE: 		0.555                                                                                                                                                            
SCORE: 		0.605                                                                                                                                                            
SCORE: 		0.550                                                                                                                                   

In [10]:

with open('study_gb_linear_features_wo_dec_scalePosWeights_1.pickle', 'wb') as handle:
    pickle.dump(trials_standard, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('params_gb_linear_features_wo_dec_scalePosWeights_1.pickle', 'wb') as handle:
    pickle.dump(best_hyperparams_standard, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
with open('study_gb_linear_features_with_dec_scalePosWeights_1.pickle', 'wb') as handle:
    pickle.dump(trials_dec, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('params_gb_linear_features_with_dec_scalePosWeights_1.pickle', 'wb') as handle:
    pickle.dump(best_hyperparams_dec, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
best_params_list = []
file_names = [('params_gb_linear_features_wo_dec_scalePosWeights_1.pickle', 'Feature_DS'), 
              ('params_gb_linear_features_with_dec_scalePosWeights_1.pickle', 'DEC_Feature_DS'), 
              ('params_gb_linear_features_wo_dec.pickle', 'Feature_DS'), 
              ('params_gb_linear_features_with_dec.pickle', 'DEC_Feature_DS')]

params={'objective': 'binary:logistic',
        'scale_pos_weight': 6,
        'updater': 'shotgun',
        'verbosity': 0,
        'booster': 'gblinear',
        'feature_selector': 'shuffle'
    }

for file_name, dataset_name in file_names:
    with (open(file_name, "rb")) as openfile:
        while True:
            try:
                best_params = pickle.load(openfile)
                
                for key, value in params.items():
                    best_params[key] = value
                    
                if file_name.find("1") != -1:
                    best_params["scale_pos_weight"] = 1
                else: 
                    best_params["scale_pos_weight"] = 6
                
                best_params_list.append((best_params, dataset_name))
            except EOFError:
                break

found 1
found 1


In [8]:
print(best_params_list)

[({'alpha': 0.6833582677557102, 'lambda': 0.7519984324901798, 'learning_rate': 0.04038678123429325, 'num_boost_round': 420.1652147086029, 'objective': 'binary:logistic', 'scale_pos_weight': 1, 'updater': 'shotgun', 'verbosity': 0, 'booster': 'gblinear', 'feature_selector': 'shuffle'}, 'Feature_DS'), ({'alpha': 0.7973402678675838, 'lambda': 1.4393908525751467, 'learning_rate': 0.028671610518432145, 'num_boost_round': 746.5644975040432, 'objective': 'binary:logistic', 'scale_pos_weight': 1, 'updater': 'shotgun', 'verbosity': 0, 'booster': 'gblinear', 'feature_selector': 'shuffle'}, 'DEC_Feature_DS'), ({'alpha': 2.2012845190671833, 'lambda': 9.585749777482139, 'learning_rate': 0.06281582374305802, 'num_boost_round': 559.4507945736697, 'objective': 'binary:logistic', 'scale_pos_weight': 6, 'updater': 'shotgun', 'verbosity': 0, 'booster': 'gblinear', 'feature_selector': 'shuffle'}, 'Feature_DS'), ({'alpha': 3.669265694367747e-06, 'lambda': 0.003691484710585189, 'learning_rate': 0.0101804927

In [9]:
results = {}

n_estimators = 100
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits)

for params, dataset_name in best_params_list:
    
    if dataset_name == 'Feature_DS':
        train, test = X_train, X_test
    elif dataset_name == 'DEC_Feature_DS':
        train, test = X_train_full, X_test_full
    
    booster_params = params

#for train, test, booster_params, dataset_name in [(X_train, X_test, best_hyperparams_standard, 'Feature_DS'), (X_train_full, X_test_full, best_hyperparams_dec, 'DEC_Feature_DS')]:
    print(f'Evaluating {dataset_name} dataset')
    datasets_results = {}
    #for booster_type, booster_params in best_params[dataset_name].items():
    booster_type = 'gblinear'
    #booster_params = best_params['Feature_DS'][booster_type]
    
    print(booster_type)
    print(booster_params)
    #print(f'Bosster of type {booster_type} with params {booster_params}')
    booster_params['eval_metric'] = 'auc'
    cv_results = {'f1': [],
                  'roc': [],
                  'pr': [],
                  'rc': [],
                  'acc': [],
                  'eval_result': []
                 }
    #if booster_type == 'gblinear':
    #    continue
    for train_index, eval_index in skf.split(train, y_train):
        x_train_split = train[train.index.isin(train_index)]
        x_eval_split = train[train.index.isin(eval_index)]
        y_train_split = y_train[train_index]
        y_eval_split = y_train[eval_index]

        eval_set = [(x_train_split, y_train_split), (x_eval_split, y_eval_split)]

        xgb_default = xgb.XGBClassifier(**booster_params, n_estimators=n_estimators)
        xgb_default.fit(x_train_split, 
                      y_train_split, 
                      eval_set=eval_set, 
                       early_stopping_rounds=5)
        #print(x_train_split)
        #print(test)

        y_pred_proba = xgb_default.predict_proba(test)[:, 1]
        y_pred_classes = xgb_default.predict(test)
        cv_results['eval_result'].append(xgb_default.evals_result())

        cv_results['f1'].append(metrics.f1_score(y_test, y_pred_classes))
        cv_results['roc'].append(metrics.roc_auc_score(y_test, y_pred_classes))
        cv_results['acc'].append(metrics.balanced_accuracy_score(y_test, y_pred_classes))
        cv_results['pr'].append(metrics.precision_score(y_test, y_pred_classes))
        cv_results['rc'].append(metrics.recall_score(y_test, y_pred_classes))

    scores = {'ROC-AUC Score': [np.mean(np.array(cv_results['roc'])), np.std(np.array(cv_results['roc']))],
              'F1 Score': [np.mean(np.array(cv_results['f1'])), np.std(np.array(cv_results['f1']))],
              'Precision': [np.mean(np.array(cv_results['pr'])), np.std(np.array(cv_results['pr']))],
              'Recall': [np.mean(np.array(cv_results['rc'])), np.std(np.array(cv_results['rc']))],
              'Balanced Accuracy': [np.mean(np.array(cv_results['acc'])), np.std(np.array(cv_results['acc']))]}
    print('########## RESULTS ##########')
    print(json.dumps(scores, indent=4))
    print()
    datasets_results[booster_type] = {'proba_predictions': y_pred_proba,
                                      'class_predictions': y_pred_classes,
                                      'scores': scores,
                                      'eval_results': cv_results['eval_result']
                                     }
    results[dataset_name] = datasets_results

    print()

Evaluating Feature_DS dataset
gblinear
{'alpha': 0.6833582677557102, 'lambda': 0.7519984324901798, 'learning_rate': 0.04038678123429325, 'num_boost_round': 420.1652147086029, 'objective': 'binary:logistic', 'scale_pos_weight': 1, 'updater': 'shotgun', 'verbosity': 0, 'booster': 'gblinear', 'feature_selector': 'shuffle'}


/Users/Paul/miniforge3/envs/xgboost/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[0]	validation_0-auc:0.58876	validation_1-auc:0.53409
[1]	validation_0-auc:0.61897	validation_1-auc:0.55929
[2]	validation_0-auc:0.64211	validation_1-auc:0.57551
[3]	validation_0-auc:0.66015	validation_1-auc:0.58861
[4]	validation_0-auc:0.67274	validation_1-auc:0.60033
[5]	validation_0-auc:0.68263	validation_1-auc:0.60884
[6]	validation_0-auc:0.69021	validation_1-auc:0.61596
[7]	validation_0-auc:0.69608	validation_1-auc:0.62136
[8]	validation_0-auc:0.70084	validation_1-auc:0.62571
[9]	validation_0-auc:0.70482	validation_1-auc:0.62958
[10]	validation_0-auc:0.70818	validation_1-auc:0.63275
[11]	validation_0-auc:0.71078	validation_1-auc:0.63522
[12]	validation_0-auc:0.71308	validation_1-auc:0.63697
[13]	validation_0-auc:0.71503	validation_1-auc:0.63906
[14]	validation_0-auc:0.71686	validation_1-auc:0.63999
[15]	validation_0-auc:0.71825	validation_1-auc:0.64101
[16]	validation_0-auc:0.71943	validation_1-auc:0.64172
[17]	validation_0-auc:0.72056	validation_1-auc:0.64300
[18]	validation_0-au

/Users/Paul/miniforge3/envs/xgboost/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11]	validation_0-auc:0.70816	validation_1-auc:0.64769
[12]	validation_0-auc:0.71041	validation_1-auc:0.65031
[13]	validation_0-auc:0.71238	validation_1-auc:0.65243
[14]	validation_0-auc:0.71419	validation_1-auc:0.65397
[15]	validation_0-auc:0.71568	validation_1-auc:0.65538
[16]	validation_0-auc:0.71710	validation_1-auc:0.65668
[17]	validation_0-auc:0.71843	validation_1-auc:0.65802
[18]	validation_0-auc:0.71949	validation_1-auc:0.65836
[19]	validation_0-auc:0.72054	validation_1-auc:0.65888
[20]	validation_0-auc:0.72147	validation_1-auc:0.65972
[21]	validation_0-auc:0.72216	validation_1-auc:0.66042
[22]	validation_0-auc:0.72291	validation_1-auc:0.66127
[23]	validation_0-auc:0.72359	validation_1-auc:0.66157
[24]	validation_0-auc:0.72426	validation_1-auc:0.66212
[25]	validation_0-auc:0.72474	validation_1-auc:0.66244
[26]	validation_0-auc:0.72528	validation_1-auc:0.66252
[27]	validation_0-auc:0.72580	validation_1-auc:0.66277
[28]	validation_0-auc:0.72622	validation_1-auc:0.66285
[29]	valid

...

[18]	validation_0-auc:0.74898	validation_1-auc:0.68548
[19]	validation_0-auc:0.75145	validation_1-auc:0.68534
[20]	validation_0-auc:0.75406	validation_1-auc:0.68595
[21]	validation_0-auc:0.75631	validation_1-auc:0.68622
[22]	validation_0-auc:0.75846	validation_1-auc:0.68627
[23]	validation_0-auc:0.76051	validation_1-auc:0.68639
[24]	validation_0-auc:0.76244	validation_1-auc:0.68659
[25]	validation_0-auc:0.76438	validation_1-auc:0.68677
[26]	validation_0-auc:0.76623	validation_1-auc:0.68716
[27]	validation_0-auc:0.76806	validation_1-auc:0.68746
[28]	validation_0-auc:0.76970	validation_1-auc:0.68744
[29]	validation_0-auc:0.77129	validation_1-auc:0.68733
[30]	validation_0-auc:0.77267	validation_1-auc:0.68783
[31]	validation_0-auc:0.77418	validation_1-auc:0.68787
[32]	validation_0-auc:0.77556	validation_1-auc:0.68756
[33]	validation_0-auc:0.77692	validation_1-auc:0.68775
[34]	validation_0-auc:0.77825	validation_1-auc:0.68787
[35]	validation_0-auc:0.77955	validation_1-auc:0.68779
[36]	valid

/Users/Paul/miniforge3/envs/xgboost/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17]	validation_0-auc:0.74938	validation_1-auc:0.64950
[18]	validation_0-auc:0.75189	validation_1-auc:0.64995
[19]	validation_0-auc:0.75438	validation_1-auc:0.65081
[20]	validation_0-auc:0.75666	validation_1-auc:0.65150
[21]	validation_0-auc:0.75884	validation_1-auc:0.65207
[22]	validation_0-auc:0.76095	validation_1-auc:0.65269
[23]	validation_0-auc:0.76300	validation_1-auc:0.65305
[24]	validation_0-auc:0.76492	validation_1-auc:0.65378
[25]	validation_0-auc:0.76676	validation_1-auc:0.65433
[26]	validation_0-auc:0.76860	validation_1-auc:0.65480
[27]	validation_0-auc:0.77025	validation_1-auc:0.65484
[28]	validation_0-auc:0.77188	validation_1-auc:0.65503
[29]	validation_0-auc:0.77350	validation_1-auc:0.65526
[30]	validation_0-auc:0.77511	validation_1-auc:0.65591
[31]	validation_0-auc:0.77666	validation_1-auc:0.65618
[32]	validation_0-auc:0.77809	validation_1-auc:0.65644
[33]	validation_0-auc:0.77941	validation_1-auc:0.65636
[34]	validation_0-auc:0.78079	validation_1-auc:0.65661
[35]	valid

/Users/Paul/miniforge3/envs/xgboost/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16]	validation_0-auc:0.74183	validation_1-auc:0.70027
[17]	validation_0-auc:0.74471	validation_1-auc:0.70024
[18]	validation_0-auc:0.74757	validation_1-auc:0.70029
[19]	validation_0-auc:0.75014	validation_1-auc:0.70049
[20]	validation_0-auc:0.75257	validation_1-auc:0.70026
[21]	validation_0-auc:0.75484	validation_1-auc:0.70010
[22]	validation_0-auc:0.75705	validation_1-auc:0.69995
[23]	validation_0-auc:0.75924	validation_1-auc:0.69944
[24]	validation_0-auc:0.76130	validation_1-auc:0.69894
########## RESULTS ##########
{
    "ROC-AUC Score": [
        0.6008881298035866,
        0.0057405877460224635
    ],
    "F1 Score": [
        0.28724187751687336,
        0.005800053381732027
    ],
    "Precision": [
        0.20495149900671886,
        0.0036402269404807145
    ],
    "Recall": [
        0.48,
        0.013030576286847296
    ],
    "Balanced Accuracy": [
        0.6008881298035866,
        0.005740587746022455
    ]
}


